# Basic Web Scraping
This notebook demonstrates basic of web scraping using Python. Picking a generic website such as GoodReads.com, we can scrape book information - being careful not to set off any detectors.


## Initializing the Database
For compatibility, we are using sqlite3 for our database. This will be sufficient to handle all the information that is collected in this short demo.

The following code establishes a connection to the database. If the database is not created, it will create it for you.

**Important:** It is not good practice including the username and password in plain-text / code.

In [1]:
import sqlite3
from sqlite3 import Error

# Configuration
dbName = "GoodReads.db"
username = ""
password = ""

def create_db(cursor):
    cursor.execute("CREATE TABLE IF NOT EXISTS Books ("
                        "url TEXT, "
                        "isbn TEXT, "
                        "name TEXT, "
                        "author TEXT, "
                        "author_url TEXT,"
                        "rating FLOAT, "
                        "rating_count INTEGER, "
                        "review_count INTEGER,"
                        "image_url TEXT,"
                        "related TEXT"
                        ")")

# Establish a connection with the database
try:
    username = username
    password = password
    connection = sqlite3.connect(dbName)
    cursor = connection.cursor()
    create_db(cursor)
except Error as e:
    print(e)
print("Database Created.")


Database Created.


## Scraping Books
This basic implementation of a scraper targets books exclusively. Note that it collects core book data that would be good for data analysis (such as interaction data). It also collects a list of related books that would allow us to create spider to continually scrape more and more books.

In [2]:
url = "https://www.goodreads.com/book/show/49628.Cloud_Atlas"

In [4]:
import re
import requests
from bs4 import BeautifulSoup
from lxml import html

# Main url parsing
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
response = html.fromstring(page.content)
item = {}
try:
    # URL data
    item['url'] = url
    item['id'] = re.search(pattern="\d+", string=item['url']).group(0)
    item['image_url'] = soup.find(id="coverImage")['src']

    # Identifying Information
    item['name'] = soup.find(id="bookTitle").text
    item['isbn'] = soup.find(itemprop="isbn").text
    item['author'] = soup.find(id="bookAuthors").find(itemprop="name").text
    item['author_url'] = soup.find(class_="authorName", href=True)['href']

    # Interaction
    item['rating'] = soup.find(itemprop="ratingValue").text
    item['rating_count'] = soup.find(itemprop="ratingCount").text
    item['review_count'] = soup.find(itemprop="reviewCount").text

    # Get related urls to scrap later
    links = []
    related = soup.find(class_="carouselRow")
    related_a = related.find_all('a', href=True)
    for r in related_a:
        if r is not None:
            links.append(r['href'])
    item['related'] = links
except AttributeError as e:
    print(e)

# Clean up the data
item['name'] = str(item['name']).strip()
item['rating'] = re.search(pattern="\d.?\d*", string=item['rating']).group(0)
item['rating_count'] = "".join(re.search(pattern="(\d*),*(\d+)", string=item['rating_count']).group(1, 2))
item['review_count'] = "".join(re.search(pattern="(\d*),*(\d+)", string=item['review_count']).group(1, 2))

print(item)

ModuleNotFoundError: No module named 'bs4'

# Adding Books to the Database
Once the information is obtained from the website, we can add it to database. 

**Important**: Connect to the database must be established before insertion of data

In [ ]:
# Add the scraped data to the database -- database must be open
sql = '''INSERT INTO Books VALUES(?,?,?,?,?,
                                  ?,?,?,?,?)'''
cursor.execute(sql, (item["url"],
                          item["isbn"],
                          item["name"],
                          item["author"],
                          item["author_url"],

                          float(item["rating"]),
                          int(item["rating_count"]),
                          int(item["review_count"]),
                          item["image_url"],
                          str(item["related"])))

connection.commit()
print("Book successfully added")

Check that the book was successfully added by querying the last row of the table.

In [ ]:
# Check the last book in the database using a query
query = "SELECT * FROM Books WHERE ROWID IN ( SELECT max( ROWID ) FROM Books );"
print(cursor.execute(query).fetchone())